In [273]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET
import numpy as np
import re

In [274]:
DATE_TEMPLATE = re.compile(r'\d{2}\.\d{2}\.\d{4}')

In [275]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s")

        remove_explicit_from_sentence(source_sentence)

        text = ''.join(source_sentence.itertext())
        dict_of_leads[id] = re.sub(r'\.$', '', text.strip());
    return dict_of_leads

def remove_explicit_from_sentence(xml_sentence):
    
    def remove_target_node(xml_sentence, target_template):
        for node in xml_sentence:
            if re.search(target_template, node.attrib["lemma"]):
                node.text = ''

    # to remove
    phoneno = re.compile(r'\d{8,12}')
    date = DATE_TEMPLATE

    remove_target_node(xml_sentence, phoneno)
    remove_target_node(xml_sentence, date)

"""
   <b>
      <s>
         21.06.2016
         <P n0="" lemma="89140580517">89140580517</P>
         хочет
         <W n1="" lemma="buy">купить</W>
         <W n2="" lemma="ноутбук">ноутбук</W>
         за 17000 руб ездить
         <P n3="" lemma="Якутия">Якутия</P>
         .
      </s>
   </b>
"""
one

'ноутбук леново g500 купить'

In [276]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

def make_common_table():
    appendix = [
        "CustomerBuys_Word",
        "CustomerSells_Word",
        "Pawn_Word",
        "Repare_Word",
        "Matter_Notebook",
        "Matter_Phone",
        "Matter_Tablet",
        "Matter_TV",
        "Matter_Other",
        "CustomerPlace_Yakutia",
        "CustomerPlace_Buryatia",
        "Communication_SMS",
        "Communication_WhatsApp",
     ]
    calls = df(columns=appendix)
    
    facts = compare_facts_to_leads()
    leads = make_dict_of_leads()

    for lead in facts:
        try:
            elems = facts[lead]
            one_sentence = leads[lead]
            cols = ["lead_id", "conversation"]
            values = [lead, one_sentence]
            for fact_name in elems:
                for fact_field in fact_name:
                    cols.append(fact_name.tag + "_" + fact_field.tag)
                    values.append(fact_field.attrib["val"])
            one_row = pd.DataFrame([values], columns=cols)
            calls = calls.append(one_row)

            values = []
            cols = []
        except ValueError as e:
            print(e)
            print(lead, leads[lead])
        except AssertionError as e:
            print(e)
            print(lead, leads[lead])
    return calls


In [277]:
calls = make_common_table()
calls["lead_id"] = calls["lead_id"].map(int)
calls = calls.sort_values(by="lead_id")

calls = calls.set_index("lead_id")


Plan shapes are not aligned
18 ультрабук продать 


In [278]:
cols_for_buy_and_sell = [
    [
        "CallDate_Date",
        "CustomerPhone_Phone",
        "Matter_Notebook",
        "Matter_Tablet",
        "Matter_Phone",        
        "Matter_TV",
        "Matter_Other",
        "conversation",
        "Комплектация",
        "Цена клиента",
        "Наша цена",
        "Утилизация",
        "Решение клиента",
    ],

    [
        "Дата",
        "Номер телефона",
        "Ноутбук/нетбук",
        "Планшет",
        "Телефон",        
        "Телевизор",
        "Другое",
        "Разговор",
        "Комплектация",
        "Цена клиента",
        "Наша цена",
        "Утилизация",
        "Решение клиента",
    ]
]

In [279]:
# Подготовка вывода для листа "Бурятия"

# todo действие по умолчанию - купить. Собрать такие графы, в которых нет действия, в таблицу "Купить" - клиент покупает у нас
# todo ремонт только телефонов и ноутбуков - объединить всё, кроме телефонов и ноутбуков в графу "Другое"


calls_from_buryatia = calls[calls.CustomerPlace_Buryatia.notnull()]
usial_matter = ["Matter_Notebook", "Matter_Phone", "Matter_Tablet"]
matter_except_usial = [col for col in calls_from_buryatia.columns if col.startswith("Matter") and col not in usial_matter]

def join_other_matters(source_df, cols, new_name):
    matter_other = df()
    for matter in cols:
        if source_df[matter].any():
            matter_other = matter_other.append(source_df[matter])
    other_buryatia = matter_other.dropna().transpose()
    # Переименовал колонку с другими предметами в "Комплектующее", т.к. оператор забивал туда
    print(other_buryatia)
    other_buryatia.columns = [new_name]
    return source_df.join(other_buryatia)

def extract_and_rename(source_df, cols):
    extracted = df(source_df, columns=cols[0])
    extracted.columns=cols[1]
    return extracted

def prepare_for_excel_ykt(source_df, col_name, output_cols):
    yakutia = source_df[source_df.CustomerPlace_Buryatia.isnull()]
    yakutia = yakutia[yakutia.Communication_SMS.isnull() & yakutia.Communication_WhatsApp.isnull()]
    actions = yakutia[yakutia[col_name].notnull()]
    return extract_and_rename(actions, output_cols)

def prepare_for_buryatia(all_calls, cols):
    # Звонки из Бурятии
    calls_from_buryatia = calls[calls.CustomerPlace_Buryatia.notnull()]
    
    #calls_buryatia_joined = join_other_matters(calls_from_buryatia, matter_except_usial, "Комплектующее")
    calls_buryatia_joined = calls_from_buryatia;
    #return extract_and_rename(calls_buryatia_joined, cols)
    return extract_and_rename(calls_from_buryatia, cols)

def prepare_for_smswhatsapp():
    cols = ["Communication_WhatsApp", "Communication_SMS"]
    calls["smswhatsapp"] = calls["Communication_WhatsApp"].append(calls["Communication_SMS"]).dropna()
    smswhatsapp = calls[calls["smswhatsapp"].notnull()]
    return extract_and_rename(smswhatsapp, cols_for_buryatia)

calls_from_buryatia

#join_other_matters(calls_from_buryatia, matter_except_usial, "Комплектующее")

,CallDate_Date,Communication_SMS,Communication_WhatsApp,CustomerBuys_Word,CustomerPhone_Phone,CustomerPlace_Buryatia,CustomerPlace_Yakutia,CustomerSells_Word,Matter_Notebook,Matter_Other,Matter_Phone,Matter_TV,Matter_Tablet,Pawn_Word,Repare_Word,conversation
lead_id,,,,,,,,,,,,,,,,
9,09.06.2016,NaN,NaN,NaN,89025620571,БУРЯТИЯ,NaN,NaN,НЕТБУК,NaN,NaN,NaN,NaN,NaN,РЕМОНТ,нетбук ремонт экран улан-удэ
31,09.06.2016,NaN,NaN,NaN,89503880906,БУРЯТИЯ,NaN,ВЫКУП,НЕТБУК,NaN,NaN,NaN,NaN,NaN,NaN,нетбук сломан продать УУ
55,12.06.2016,NaN,NaN,NaN,89021661922,БУРЯТИЯ,NaN,ВЫКУП,НОУТБУК,NaN,NaN,NaN,NaN,NaN,NaN,ноутбук продать уу


In [280]:
cols_for_buryatia = [
    [
        "CallDate_Date",
        "CustomerPhone_Phone",
        "Matter_Notebook",
        "Matter_Tablet",
        "Matter_Phone",
        "Комплектующее",
        "conversation",
        "Комплектация",
        "Цена клиента",
        "Наша цена",
        "Утилизация",
        "Решение клиента",
    ],

    [
        "Дата",
        "Номер телефона",
        "Ноутбук/нетбук",
        "Планшет",
        "Телефон",
        "Другое",
        "Разговор",
        "Комплектация",
        "Цена клиента",
        "Наша цена",
        "Утилизация",
        "Решение клиента",
    ]
]

In [281]:
writer = pd.ExcelWriter('report.xlsx')

vikup = prepare_for_excel_ykt(calls, "CustomerSells_Word", cols_for_buy_and_sell)
prodazha = prepare_for_excel_ykt(calls, "CustomerBuys_Word", cols_for_buy_and_sell)
remont = prepare_for_excel_ykt(calls, "Repare_Word", cols_for_buy_and_sell)
lombard = prepare_for_excel_ykt(calls, "Pawn_Word", cols_for_buy_and_sell)
buryatia = prepare_for_buryatia(calls, cols_for_buryatia)
smswhatsapp = prepare_for_smswhatsapp()

vikup.to_excel(writer, sheet_name = "Выкуп", index=False)
prodazha.to_excel(writer, sheet_name = "Продажа", index=False)
remont.to_excel(writer, sheet_name = "Ремонт", index=False)
lombard.to_excel(writer, sheet_name = "Ломбард", index=False)
buryatia.to_excel(writer, sheet_name = "Бурятия", index=False)
smswhatsapp.to_excel(writer, sheet_name = "smswhatsapp", index=False)
writer.save()